In [1]:
# import libraries 
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

#from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img

from imutils import paths
import cv2 as cv
import numpy as np
import time
import os

Using TensorFlow backend.


In [2]:
def spiralForm(m, n, a) : 
    k = 0; l = 0
  
    ''' k - starting row index 
        m - ending row index 
        l - starting column index 
        n - ending column index 
        i - iterator '''
      
  
    while (k < m and l < n) : 
          
        # append the first row from the remaining rows  
        for i in range(l, n) : 
            reverse_spiral.append(a[k][i])
              
        k += 1
  
        # append the last column from the remaining columns  
        for i in range(k, m) : 
            reverse_spiral.append(a[i][n - 1])
              
        n -= 1
  
        # append the last row from the remaining rows  
        if ( k < m) : 
            for i in range(n - 1, (l - 1), -1) : 
                reverse_spiral.append(a[m - 1][i])
              
            m -= 1
          
        # append the first column from the remaining columns  
        if (l < n) : 
            for i in range(m - 1, k - 1, -1) : 
                reverse_spiral.append(a[i][l])
              
            l += 1

In [3]:
def rev_spiral(r,c,window,reverse_spiral):

    window_size = window
    rows = r
    cols = c
    a=np.zeros((rows,cols,2))
    count=0
    for l,i in zip(range(0,rows,1),range(window_size,height+1,window_size)):
        for m,j in zip(range(0,cols,1),range(window_size,width+1,window_size)):
            a[l,m]=[i,j]

    a=a.astype(int)
    spiralForm(rows, cols, a) 
    reverse_spiral.reverse()
    reverse_spiral = np.array(reverse_spiral)    

In [ ]:
# import re
# def sorted_aphanumeric(data):
#     convert = lambda text: int(text) if text.isdigit() else text.lower()
#     alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
#     return sorted(data, key=alphanum_key)

In [4]:
# parameters

inputPath ="InputImages/"
outputPath="OutputImages/"
imagelistInput = os.listdir(inputPath)
# imagePaths = sorted_aphanumeric(imagePaths)
monkey =["chimpanzee","gorilla","guenon","patas","baboon","macaque","langur","colobus","proboscis_monkey","marmoset","capuchin","howler_monkey","titi","spider_monkey","squirrel_monkey","guenon_monkey","hussar_monkey","Erythrocebus_patas","colobus_monkey","Nasalis_larvatus","ringtail","Cebus_capucinus","howler","titi_monkey","Ateles_geoffroyi","Saimiri_sciureus"]

predicted_list =[]
time_taken =[]

# load model
model = VGG19()
print(model.summary())

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
# Process inputs

#create a plot of the layers in the VGG mode  
for file_name in imagelistInput:
    print(file_name)
    
    # load an image from file
    img = cv.imread(inputPath+file_name)
    tok =time.time() 
    reverse_spiral=[]
    height = img.shape[0]
    width = img.shape[1]
    count=0
    window = 224
    rows = int(height/224)
    cols = int(width/224)
    flag=0
    rev_spiral(rows,cols,window,reverse_spiral)
    
    arr3d = np.reshape(reverse_spiral, (rows, cols, 2))
    
    for i in range(0,rows,1):
        for j in range(0,cols,1):
            image = img[arr3d[i][j][0]-window:arr3d[i][j][0],arr3d[i][j][1]-window:arr3d[i][j][1]]
            count+=1

            # convert the image pixels to a numpy array
            image = img_to_array(image)
            # reshape data for the model
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            # prepare the image for the VGG model
            image = preprocess_input(image)
            # predict the probability across all output classes
            yhat = model.predict(image)
            # convert the probabilities to class labels
            label = decode_predictions(yhat)
            # retrieve the most likely result, e.g. highest probability
            label = label[0][0]
            if(label[2]>0.1):
                if(label[1] in monkey):
                    predicted_list.append(1)
                    flag=1
                    print("Monkey detected")
                    break
        if(flag==1):
            break
    if(flag==0):
        predicted_list.append(0)
    tik=time.time()
    time_taken.append('%.3f'%(tik-tok))
    print("Time : ", '%.3f'%(tik-tok),"\n")

print(predicted_list, time_taken)

16b5881aa0469a10.jpg
Monkey detected
Time :  1.198 

monkey1.jpg
Monkey detected
Time :  6.488 

monkey_s.jpg
Monkey detected
Time :  2.993 

test.jpeg
Time :  28.036 

yolo_out_py17.jpg
Monkey detected
Time :  1.960 

[1, 1, 1, 0, 1] ['1.198', '6.488', '2.993', '28.036', '1.960']
